In [1]:
import os
import pickle

import keras.backend as K
import keras.utils
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from keras.applications import Xception
from keras.layers import Input, Activation, Flatten, Dense
from keras.layers import (concatenate)
from keras.models import Model
from keras.optimizers import Adam
from skimage.transform import resize
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm



data_image = "C:\\Users\\Priit\\Dropbox\\Informaatika\\Magister\\Tehisnärvivõrgud\\data\\train_images"
data_dir = "C:\\Users\\Priit\\Dropbox\\Informaatika\\Magister\\Tehisnärvivõrgud\\data"

label_column = "AdoptionSpeed"

BATCH_SIZE = 256 * 2

test_size = 0.2
height, width = 100, 100

Using TensorFlow backend.


In [2]:
# Source: https://www.kaggle.com/christofhenkel/weighted-kappa-loss-for-keras-tensorflow
def kappa_loss(y_pred, y_true, y_pow=2, eps=1e-3, N=5, bsize=BATCH_SIZE, name='kappa'):
    """A continuous differentiable approximation of discrete kappa loss.
        Args:
            y_pred: 2D tensor or array, [batch_size, num_classes]
            y_true: 2D tensor or array,[batch_size, num_classes]
            y_pow: int,  e.g. y_pow=2
            N: typically num_classes of the model
            bsize: batch_size of the training or validation ops
            eps: a float, prevents divide by zero
            name: Optional scope/name for op_scope.
        Returns:
            A tensor with the kappa loss."""

    with tf.name_scope(name):
        y_true = tf.to_float(y_true)
        repeat_op = tf.to_float(tf.tile(tf.reshape(tf.range(0, N), [N, 1]), [1, N]))
        repeat_op_sq = tf.square((repeat_op - tf.transpose(repeat_op)))
        weights = repeat_op_sq / tf.to_float((N - 1) ** 2)
    
        pred_ = y_pred ** y_pow
        try:
            pred_norm = pred_ / (eps + tf.reshape(tf.reduce_sum(pred_, 1), [-1, 1]))
        except Exception:
            pred_norm = pred_ / (eps + tf.reshape(tf.reduce_sum(pred_, 1), [bsize, 1]))
    
        hist_rater_a = tf.reduce_sum(pred_norm, 0)
        hist_rater_b = tf.reduce_sum(y_true, 0)
    
        conf_mat = tf.matmul(tf.transpose(pred_norm), y_true)
    
        nom = tf.reduce_sum(weights * conf_mat)
        denom = tf.reduce_sum(weights * tf.matmul(
            tf.reshape(hist_rater_a, [N, 1]), tf.reshape(hist_rater_b, [1, N])) /
                              tf.to_float(bsize))
    
        return nom / (denom + eps)

In [3]:
train = pd.read_csv(os.path.join(data_dir, "train.csv"), sep=',')
pet_ids = train["PetID"]

selected_columns = ["Type",
                    "Gender",
                    "Color1",
                    "Color2",
                    "Color3",
                    "MaturitySize",
                    "FurLength",
                    "Vaccinated",
                    "Dewormed",
                    "Sterilized",
                    "Health",
                    "State",
                    "MaturitySize"]




y = train[label_column]

# One-Hot-encode
X = pd.get_dummies(train[selected_columns], columns=selected_columns)

# Normalize:
to_normalize = ["Age", "Fee", "Quantity"]
for to_norm in to_normalize:
     X[to_norm] = (train[to_norm] - train[to_norm].mean()) / train[to_norm].std()

In [4]:
f_im_name = "images.binary"

if not os.path.isfile(f_im_name):
    
    image_paths = [os.path.join(data_dir, data_image, pet_id + "-1.jpg") for pet_id in pet_ids]

    images = []

    for path in tqdm(image_paths):
        if os.path.isfile(path):
            image = Image.open(path).convert("RGB") 
            image.load()
            image = np.asarray(image, dtype="int32" )
            image = resize(image, (height, width), anti_aliasing=True, mode='constant')
        else:
            image = np.random.rand(height, width, 3) * 255
        
        images.append(image)
    
    images = np.array(images)

    # Standardize:
    mean = np.mean(images)
    std = np.std(images)

    images_meanstd = (images - mean)/std
    with open(f_im_name, 'wb') as handle:
        pickle.dump(images_meanstd, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(f_im_name, 'rb') as handle:
        images_meanstd = pickle.load(handle)

print(images_meanstd.shape)

(14993, 100, 100, 3)


In [5]:
X_train_img, X_test_img, X_train_else, X_test_else, y_train, y_test = train_test_split(images_meanstd, 
                                                                                       X, 
                                                                                       y, 
                                                                                       test_size=test_size,
                                                                                       random_state=1)

In [6]:
# https://stackoverflow.com/questions/49618986/neural-network-in-keras-with-two-different-input-types-images-and-values
# https://www.learnopencv.com/keras-tutorial-transfer-learning-using-pre-trained-models/

transfer = Xception(weights='imagenet', include_top=False, input_shape=(height, width, 3))

# Freeze Xception
for layer in transfer.layers[:-3]:
    layer.trainable = False

# Inputs
image_input = Input(shape=(height, width, 3))
aux_input = Input(shape=(len(list(X_train_else)),))

# Images:
transfer = transfer(image_input)
transfer = Dense(150, activation='relu')(transfer)
flatten = Flatten()(transfer)

# Aux input:
x = Dense(150, activation='relu')(aux_input)
x = Dense(250, activation='relu')(x)
x = Dense(350, activation='relu')(x)

# Merged:
merge = concatenate([flatten, x])
x = Dense(500)(merge)
x = Dense(450, activation='relu')(x)
x = Dense(100, activation='relu')(x)
h = Dense(5)(x)

# Predictions:
predictions = Activation('softmax')(h)

def kappa(y_true, y_pred):
    y_pred = map(lambda x: np.argmax(x), y_pred) 
    return cohen_kappa_score(y_true, y_pred, weights="quadratic")

model = Model(inputs=[image_input, aux_input], outputs=predictions)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit([X_train_img, X_train_else], 
                    keras.utils.to_categorical(y_train),
                    batch_size=BATCH_SIZE, 
                    epochs=10, 
                    validation_split=0.1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 10794 samples, validate on 1200 samples
Epoch 1/10
10794/10794 [==============================] - ETA: 22:30 - loss: 1.6541 - acc: 0.11 - ETA: 22:50 - loss: 1.6318 - acc: 0.19 - ETA: 20:26 - loss: 1.6116 - acc: 0.21 - ETA: 18:32 - loss: 1.6319 - acc: 0.22 - ETA: 17:02 - loss: 1.6187 - acc: 0.25 - ETA: 15:41 - loss: 1.5946 - acc: 0.26 - ETA: 1:35:40 - loss: 1.5793 - acc: 0.27 - ETA: 1:19:55 - loss: 1.5770 - acc: 0.27 - ETA: 1:07:39 - loss: 1.5754 - acc: 0.27 - ETA: 57:33 - loss: 1.5676 - acc: 0.2830 - ETA: 48:58 - loss: 1.5621 - acc: 0.28 - ETA: 41:36 - loss: 1.5548 - acc: 0.28 - ETA: 35:10 - loss: 1.5484 - acc: 0.29 - ETA: 29:29 - loss: 1.5420 - acc: 0.29 - ETA: 24:19 - loss: 1.5359 - acc: 0.29 - ETA: 19:35 - loss: 1.5301 - acc: 0.29 - ETA: 15:15 - loss: 1.5249 - acc: 0.29 - ETA: 11:09 - loss: 1.5188 - acc: 0.29 - ETA: 7:19 - loss: 1.5144 - acc: 0.2975 - ETA:

10794/10794 [==============================] - ETA: 24:06 - loss: 1.3071 - acc: 0.43 - ETA: 21:39 - loss: 1.3107 - acc: 0.43 - ETA: 20:14 - loss: 1.3193 - acc: 0.43 - ETA: 18:57 - loss: 1.3067 - acc: 0.43 - ETA: 17:45 - loss: 1.3083 - acc: 0.43 - ETA: 17:03 - loss: 1.3092 - acc: 0.43 - ETA: 16:18 - loss: 1.3092 - acc: 0.42 - ETA: 15:25 - loss: 1.3102 - acc: 0.43 - ETA: 14:14 - loss: 1.3098 - acc: 0.43 - ETA: 12:59 - loss: 1.3095 - acc: 0.43 - ETA: 11:48 - loss: 1.3110 - acc: 0.43 - ETA: 10:35 - loss: 1.3091 - acc: 0.43 - ETA: 9:19 - loss: 1.3059 - acc: 0.4330 - ETA: 8:03 - loss: 1.3057 - acc: 0.430 - ETA: 6:50 - loss: 1.3071 - acc: 0.428 - ETA: 5:39 - loss: 1.3099 - acc: 0.428 - ETA: 4:30 - loss: 1.3095 - acc: 0.429 - ETA: 3:22 - loss: 1.3084 - acc: 0.430 - ETA: 2:16 - loss: 1.3081 - acc: 0.431 - ETA: 1:11 - loss: 1.3106 - acc: 0.427 - ETA: 5s - loss: 1.3108 - acc: 0.4281  - 3592s 333ms/step - loss: 1.3107 - acc: 0.4285 - val_loss: 1.4366 - val_acc: 0.3575
Epoch 9/10
10794/10794 [=====

In [7]:
train_pred = [np.argmax(pred) for pred in model.predict([X_train_img, X_train_else])]
test_predictions = [np.argmax(pred) for pred in model.predict([X_test_img, X_test_else])]

In [8]:
print("Kappa on train: {}".format(round(cohen_kappa_score(y_train, train_pred, weights="quadratic"), 4)))
print("Accuracy on train: {}".format(round(accuracy_score(y_train, train_pred), 4)))
print("________________")
print("Kappa on test: {}".format(round(cohen_kappa_score(y_test, test_predictions, weights="quadratic"), 4)))
print("Accuracy on test: {}".format(round(accuracy_score(y_test, test_predictions), 4)))

Kappa on train: 0.377
Accuracy on train: 0.447
________________
Kappa on test: 0.2542
Accuracy on test: 0.3525


In [9]:
model.save('test_model.h5')